In [0]:


import numpy as np
import pandas as pd
import tensorflow as tf
from os.path import join
from os import listdir
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm
from sklearn.metrics import precision_score,jaccard_similarity_score,confusion_matrix
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau,CSVLogger
from keras import utils
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as k
from keras import losses 
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

path='/gdrive/My Drive/Colab Notebooks/ML4HC/project4/'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


Using TensorFlow backend.


## Functions

In [0]:
def get_data(path_images,path_labeled_images):
  images=[]
  labeled_images=[]
  for file in listdir(path_images):
      image = np.load(join(path_images, file))
      label = np.load(join(path_labeled_images, file))
      images.append(image)
      labeled_images.append(label)
  return images,labeled_images

In [0]:
def get_2dimages(images,labels):
  X=[]
  Y=[]
  n_images=len(images)
  for im in range(n_images):
    for depth in range(images[im].shape[0]):
      X.append(images[im][depth])
      Y.append(labels[im][depth])
  return np.expand_dims(np.array(X),-1),np.expand_dims(np.array(Y),-1)

    

In [0]:
 def get_augmented_images(X,y,rotation_range=100,times_training_set=3,save_path_X=path+'rot_images',save_path_y=path+'rot_labels'):
  
  train_gen=ImageDataGenerator(rotation_range=rotation_range)
  mask_gen=ImageDataGenerator(rotation_range=rotation_range)

  train_gen.fit(X,seed=1)
  mask_gen.fit(y,seed=1)
  i=0
  X_rotated=[]
  for X_batch in train_gen.flow(X,seed=1, batch_size=1):
      X_rotated.append(X_batch)
      i+=1
      if i==times_training_set*X.shape[0]:break

  X_rot=np.array(X_rotated).reshape(-1,256,256,1) 
  np.save(save_path_X,X_rot)

  i=0
  y_rotated=[]
  for y_batch in tqdm(mask_gen.flow(y,seed=1, batch_size=1)):
      y_rotated.append(y_batch)
      i+=1
      if i==times_training_set*y.shape[0]:break
  y_rot=np.array(y_rotated).reshape(-1,256,256,1)
  np.save(save_path_y,y_rot)
  
  return X_rot,y_rot

In [0]:
#normaliztion
def normalize(X):
  "3d images as input"
  means=X.mean(axis=(1,2))
  stds=X.std(axis=(1,2))

  means_=np.repeat(means,X.shape[1]*X.shape[2]).reshape(-1,X.shape[1],X.shape[2],1)
  stds_=np.repeat(stds,X.shape[1]*X.shape[2]).reshape(-1,X.shape[1],X.shape[2],1)

  X_norm=(X-means_)/stds_
  return X_norm

In [0]:
def avg_precision(y_true,y_pred,how):
  prec=[]
  y_true=np.argmax(y_true,axis=-1)
  for im in range(y_true.shape[0]):
    prec_score=precision_score(y_true[im].ravel(),y_pred[im].ravel(),average=how)
    prec.append(prec_score)
  return np.mean(prec),prec

def avg_Iou(y_true,y_pred):
  y_true=np.argmax(y_true,axis=-1)
  
  ious=[]
  for im in range(y_true.shape[0]):
    confmat=confusion_matrix(y_true[im].ravel(),y_pred[im].ravel(),labels=[0,1,2])
    Iou=np.divide(np.diag(confmat), np.sum(confmat, axis=1) + np.sum(confmat, axis=0) - np.diag(confmat)+1e-100)
    ious.append(np.average(Iou))
    
  return np.mean(ious),ious
  

def get_metrics(y_true,y_pred,path=path+'results.csv',return_scores=False):
  
  avg_micro,_=avg_precision(y_true,y_pred,'micro')
  avg_macro,_=avg_precision(y_true,y_pred,'macro')
  avg_weighted,_=avg_precision(y_true,y_pred,'weighted')
  avg_iou,_=avg_Iou(y_true,y_pred)
  dict_={"Avg_micro":[avg_micro],"Avg_macro":[avg_macro],"Avg_weigted_macro":[avg_weighted],"Avg_Iou":[avg_iou]}
  results=pd.DataFrame(dict_)
  results.to_csv(path)
  if return_scores:
    return dict
  
  print(f"Scores: Avg_micro={round(avg_micro,3)},Avg_macro={round(avg_macro,3)},Avg_weigted_macro={round(avg_weighted,3)},Avg_Iou={round(avg_iou,3)}")

In [0]:
img,lbl=get_data(path+'Data/'+'train_images',path+'Data/'+'train_labels')
X,y=get_2dimages(img,lbl)

In [0]:
#Get Rotated Images
X_rot,y_rot=get_augmented_images(X,y,rotation_range=100,times_training_set=6,save_path_X=path+'rot_images',save_path_y=path+'rot_labels')

In [0]:
X=np.concatenate((X,X_rot),axis=0)
Y=np.concatenate((y,y_rot),axis=0)

In [0]:
X_norm=normalize(X)

In [0]:
X_train,X_val,y_train,y_val=train_test_split(X_norm,Y,test_size=0.2,random_state=42)



In [0]:
y_train=utils.to_categorical(y_train,num_classes=3)


In [0]:
y_val=utils.to_categorical(y_val)

## Unet

In [0]:
def unet2d( n_classes=3 , input_shape=(256,256,1)):

	
	image_input = Input(shape=input_shape)


	conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)
	conv1 = Dropout(0.2)(conv1)
	conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv1)
	pool1 = MaxPooling2D((2, 2))(conv1)
	
	conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
	conv2 = Dropout(0.2)(conv2)
	conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv2)
	pool2 = MaxPooling2D((2, 2))(conv2)
	conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
	conv3 = Dropout(0.2)(conv3)
	conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(conv3)
  
	up1 = concatenate([UpSampling2D((2, 2))(conv3), conv2], axis=-1)
	conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
	conv4 = Dropout(0.2)(conv4)
	conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv4)
	
	up2 = concatenate([UpSampling2D((2, 2))(conv4), conv1], axis=-1)
	conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
	conv5 = Dropout(0.2)(conv5)
	conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv5)
	
	output = Conv2D( n_classes, (1, 1) , activation='softmax',padding='same')(conv5)

	model = Model(image_input , output )
	
	return model


In [0]:
model=unet2d()
model.compile(optimizer = Adam(), loss = losses.categorical_crossentropy)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
file_path = path+'Unet_mini_images6.h5'
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
csv_logger = CSVLogger(path+'unet_images6.txt', append=True, separator=';')
early = EarlyStopping(monitor="val_loss", mode="min", patience=5, verbose=1)
redonplat = ReduceLROnPlateau(monitor="val_loss", mode="min", patience=3, verbose=2)
callbacks_list = [checkpoint,early,redonplat,csv_logger]
#model.fit(x=X_train,y=y_train,batch_size=32,epochs=50,verbose=2,validation_data=(X_val,y_val),callbacks=callbacks_list)

## Tests

In [0]:
model.load_weights(file_path)

### Normal Images


In [0]:
img_test,lbl_test=get_data(path+'Data/'+'test_images',path+'Data/'+'test_labels')


In [0]:
#check test set
for im in range(len(img_test)):
  print("Shape Image",im+1,img_test[im].shape,lbl_test[im].shape)

  


Shape Image 1 (15, 256, 256) (15, 256, 256)
Shape Image 2 (14, 256, 256) (14, 256, 256)
Shape Image 3 (21, 432, 432) (21, 432, 432)
Shape Image 4 (20, 256, 256) (20, 256, 256)
Shape Image 5 (19, 256, 256) (19, 256, 256)
Shape Image 6 (17, 256, 256) (17, 256, 256)
Shape Image 7 (15, 256, 256) (15, 256, 256)
Shape Image 8 (19, 256, 256) (19, 256, 256)
Shape Image 9 (20, 256, 256) (20, 256, 256)
Shape Image 10 (15, 256, 256) (15, 256, 256)


In [0]:
#image 3 has different shape
#crop it wisely
#cut
#70 up,106 down
#90 left,86 right
img_test[2]=img_test[2][:,70:-106,90:-86]
lbl_test[2]=lbl_test[2][:,70:-106,90:-86]

In [0]:
#Get 2d slices
X_test,y_test=get_2dimages(img_test,lbl_test)

In [0]:
#predictions

images=normalize(X_test)
y=utils.to_categorical(y_test,num_classes=3)
y_pred=model.predict(images)
y_pred=np.argmax(y_pred,axis=-1)

In [0]:
for im in range(50):
  f, (ax1, ax2) = plt.subplots(1,2 )
  ax1.imshow(y_test[im].reshape(256,256))
  ax2.imshow(y_pred[im])
  

In [114]:


#metrics
_,micros=avg_precision(y,y_pred,how='micro')
_,macros=avg_precision(y,y_pred,how='macro')
_,weighted=avg_precision(y,y_pred,how='weighted')
_,Ios=avg_Iou(y,y_pred)

# list of lists with metrics per image ->reshape metrics results
im=0
precs_micro=[]
precs_macro=[]
Ious=[]
for ind in range(len(img_test)):
  nxt_im=len(img_test[ind])
  precs_micro.append(micros[im:im+nxt_im])
  precs_macro.append(macros[im:im+nxt_im])
  Ious.append(Ios[im:im+nxt_im])
  im+=nxt_im

#make index for Result file

depths=[]
count=0
for i in range(len(img_test)):
  len_im3d=len(img_test[i])
  for _ in range(len_im3d):
    count+=1
    depths.append('depth'+str(count))
  count=0
  
images_index=[]
for i in range(len(img_test)):
  for _ in range(len(img_test[i])):
    images_index.append('image '+str(50+i))

arrays=[images_index,depths]
index = pd.MultiIndex.from_tuples(list(zip(*arrays)))

#Dataframe of results

Results=pd.DataFrame(index=index)  
Results['Overall_Precision']=[depth for im3d in precs_micro for depth in im3d] 
Results['Macro_Precision']=[depth for im3d in precs_macro for depth in im3d]
Results['Iou']=[depth for im3d in Ious for depth in im3d]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
Averages=Results.groupby(level=0,sort=False).mean()
Averages.columns=[['AVG Overall Precision', 'AVG Macro Precision', 'AVG Iou']]

In [0]:
Averages

,AVG Overall Acurracy,AVG Macro Acurracy,AVG Iou
image 50,0.918233,0.515104,0.368338
image 51,0.972610,0.663899,0.587912
image 52,0.905579,0.660889,0.466994
image 53,0.947449,0.541851,0.403202
image 54,0.963221,0.522204,0.431666
image 55,0.973159,0.746210,0.566386
image 56,0.955438,0.469150,0.371534
image 57,0.974344,0.685895,0.492354
image 58,0.940137,0.529670,0.375527
image 59,0.959805,0.692806,0.571320


In [116]:
Results

Overall_Precision  Macro_Precision       Iou
image 50 depth1            0.878922         0.333333  0.292974
         depth2            0.888916         0.333333  0.296305
         depth3            0.905869         0.444817  0.360755
         depth4            0.923492         0.324271  0.309227
         depth5            0.919601         0.570314  0.393091
         depth6            0.900208         0.410364  0.304530
         depth7            0.906250         0.796894  0.385483
         depth8            0.888123         0.296054  0.296041
         depth9            0.882538         0.798197  0.306810
         depth10           0.892548         0.781619  0.330803
         depth11           0.933136         0.767309  0.596632
         depth12           0.924820         0.644108  0.538312
         depth13           0.943954         0.559278  0.452407
         depth14           0.996338         0.333333  0.332113
         depth15           0.988785         0.333333  0.329595
image 51 depth1            0.997025         0.333333  0.332342
         depth2            0.954727         0.333333  0.318242
         depth3            0.968094         0.646020  0.609579
         depth4            0.961914         0.681746  0.644525
         depth5            0.967789         0.832817  0.698561
         depth6            0.971497         0.855940  0.725140
         depth7            0.972137         0.863886  0.744548
         depth8            0.976715         0.892237  0.784874
         depth9            0.978210         0.891625  0.795166
         depth10           0.971344         0.909191  0.783230
         depth11           0.960617         0.785635  0.661611
         depth12           0.965820         0.602155  0.476063
         depth13           0.987350         0.333333  0.329117
         depth14           0.983307         0.333333  0.327769
image 52 depth1            0.959122         0.333333  0.319707
...                             ...              ...       ...
image 58 depth6            0.942505         0.480866  0.387258
         depth7            0.934219         0.746022  0.365961
         depth8            0.930908         0.715851  0.412023
         depth9            0.924301         0.584246  0.390130
         depth10           0.906219         0.636942  0.321468
         depth11           0.903488         0.542512  0.333042
         depth12           0.904785         0.952079  0.322828
         depth13           0.907486         0.502777  0.302713
         depth14           0.936737         0.595627  0.375065
         depth15           0.939056         0.495035  0.344915
         depth16           0.971100         0.554514  0.475956
         depth17           0.976440         0.510077  0.492928
         depth18           0.970261         0.461867  0.444920
         depth19           0.944397         0.333333  0.314799
         depth20           0.937500         0.333333  0.312500
image 59 depth1            0.981201         0.333333  0.327067
         depth2            0.965195         0.333333  0.321732
         depth3            0.958008         0.589360  0.439668
         depth4            0.969498         0.886392  0.716307
         depth5            0.971329         0.904190  0.767929
         depth6            0.968460         0.942209  0.760542
         depth7            0.973190         0.912757  0.801774
         depth8            0.948853         0.837108  0.684958
         depth9            0.934402         0.762802  0.579576
         depth10           0.942749         0.807517  0.624660
         depth11           0.942184         0.814737  0.616550
         depth12           0.956497         0.776095  0.631838
         depth13           0.955688         0.825597  0.653929
         depth14           0.961990         0.333333  0.320663
         depth15           0.967834         0.333333  0.322611

[175 rows x 3 columns]

In [0]:
data={'Total AVG Overall Precision':[Averages['AVG Overall Precision'].mean()[0]],\
      'Total AVG Per Class Precision':[Averages['AVG Macro Precision'].mean()[0]],\
      'Total AVG Iou':[Averages['AVG Iou'].mean()[0]]}
total_averages=pd.DataFrame(data=data,index=['Unet2d'])

In [109]:
total_averages

,Total AVG Overall Precision,Total AVG Per Class Precision,Total AVG Iou
Unet2d,0.950998,0.602768,0.463523


### Rotated Images

In [0]:
img_test_r,lbl_test_r=get_data(path+'Data/'+'test_images_randomly_rotated',path+'Data/'+'test_labels_randomly_rotated')

In [0]:
#check test set
for im in range(len(img_test_r)):
  print("Shape Image",im+1,img_test_r[im].shape,lbl_test_r[im].shape)

  


Shape Image 1 (15, 256, 256) (15, 256, 256)
Shape Image 2 (14, 256, 256) (14, 256, 256)
Shape Image 3 (21, 432, 432) (21, 432, 432)
Shape Image 4 (20, 256, 256) (20, 256, 256)
Shape Image 5 (19, 256, 256) (19, 256, 256)
Shape Image 6 (17, 256, 256) (17, 256, 256)
Shape Image 7 (15, 256, 256) (15, 256, 256)
Shape Image 8 (19, 256, 256) (19, 256, 256)
Shape Image 9 (20, 256, 256) (20, 256, 256)
Shape Image 10 (15, 256, 256) (15, 256, 256)


In [0]:
#image 3 has different shape
#crop it wisely
#cut
#70 up,106 down
#90 left,86 right

img_test_r[2]=img_test_r[2][:,70:-106,90:-86]
lbl_test_r[2]=lbl_test_r[2][:,70:-106,90:-86]
X_test_r,y_test_r=get_2dimages(img_test_r,lbl_test_r)

In [0]:
#Get 2d slices
X_test_r,y_test_r=get_2dimages(img_test_r,lbl_test_r)

In [0]:
#predictions

images_r=normalize(X_test_r)
y_r=utils.to_categorical(y_test_r,num_classes=3)
y_pred_r=model.predict(images_r)
y_pred_r=np.argmax(y_pred_r,axis=-1)

#metrics
_,micros_r=avg_precision(y_r,y_pred_r,how='micro')
_,macros_r=avg_precision(y_r,y_pred_r,how='macro')
_,weighted_r=avg_precision(y_r,y_pred_r,how='weighted')
_,Ios_r=avg_Iou(y,y_pred)

# list of lists with metrics per image ->reshape metrics results
im=0
precs_micro_r=[]
precs_macro_r=[]
Ious_r=[]
for ind in range(len(img_test_r)):
  nxt_im=len(img_test_r[ind])
  precs_micro_r.append(micros_r[im:im+nxt_im])
  precs_macro_r.append(macros_r[im:im+nxt_im])
  Ious_r.append(Ios_r[im:im+nxt_im])
  im+=nxt_im

#make index for Result file

depths=[]
count=0
for i in range(len(img_test_r)):
  len_im3d=len(img_test_r[i])
  for _ in range(len_im3d):
    count+=1
    depths.append('depth'+str(count))
  count=0
  
images_index=[]
for i in range(len(img_test_r)):
  for _ in range(len(img_test_r[i])):
    images_index.append('image '+str(50+i))

arrays=[images_index,depths]
index = pd.MultiIndex.from_tuples(list(zip(*arrays)))

#Dataframe of results

Results_r=pd.DataFrame(index=index)  
Results_r['Overall_acurracy']=[depth for im3d in precs_micro for depth in im3d] 
Results_r['Macro_acurracy']=[depth for im3d in precs_macro for depth in im3d]
Results_r['Iou']=[depth for im3d in Ious for depth in im3d]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
Averages_r=Results_r.groupby(level=0,sort=False).mean()
Averages_r.columns=[['AVG Overall Precision', 'AVG Macro Precision', 'AVG Iou']]

In [112]:
Averages_r

,AVG Overall Precision,AVG Macro Precision,AVG Iou
image 50,0.918233,0.515104,0.368338
image 51,0.972610,0.663899,0.587912
image 52,0.905579,0.660889,0.466994
image 53,0.947449,0.541851,0.403202
image 54,0.963221,0.522204,0.431666
image 55,0.973159,0.746210,0.566386
image 56,0.955438,0.469150,0.371534
image 57,0.974344,0.685895,0.492354
image 58,0.940137,0.529670,0.375527
image 59,0.959805,0.692806,0.571320


In [115]:
Results_r

Overall_Precision  Macro_Precision       Iou
image 50 depth1            0.878922         0.333333  0.292974
         depth2            0.888916         0.333333  0.296305
         depth3            0.905869         0.444817  0.360755
         depth4            0.923492         0.324271  0.309227
         depth5            0.919601         0.570314  0.393091
         depth6            0.900208         0.410364  0.304530
         depth7            0.906250         0.796894  0.385483
         depth8            0.888123         0.296054  0.296041
         depth9            0.882538         0.798197  0.306810
         depth10           0.892548         0.781619  0.330803
         depth11           0.933136         0.767309  0.596632
         depth12           0.924820         0.644108  0.538312
         depth13           0.943954         0.559278  0.452407
         depth14           0.996338         0.333333  0.332113
         depth15           0.988785         0.333333  0.329595
image 51 depth1            0.997025         0.333333  0.332342
         depth2            0.954727         0.333333  0.318242
         depth3            0.968094         0.646020  0.609579
         depth4            0.961914         0.681746  0.644525
         depth5            0.967789         0.832817  0.698561
         depth6            0.971497         0.855940  0.725140
         depth7            0.972137         0.863886  0.744548
         depth8            0.976715         0.892237  0.784874
         depth9            0.978210         0.891625  0.795166
         depth10           0.971344         0.909191  0.783230
         depth11           0.960617         0.785635  0.661611
         depth12           0.965820         0.602155  0.476063
         depth13           0.987350         0.333333  0.329117
         depth14           0.983307         0.333333  0.327769
image 52 depth1            0.959122         0.333333  0.319707
...                             ...              ...       ...
image 58 depth6            0.942505         0.480866  0.387258
         depth7            0.934219         0.746022  0.365961
         depth8            0.930908         0.715851  0.412023
         depth9            0.924301         0.584246  0.390130
         depth10           0.906219         0.636942  0.321468
         depth11           0.903488         0.542512  0.333042
         depth12           0.904785         0.952079  0.322828
         depth13           0.907486         0.502777  0.302713
         depth14           0.936737         0.595627  0.375065
         depth15           0.939056         0.495035  0.344915
         depth16           0.971100         0.554514  0.475956
         depth17           0.976440         0.510077  0.492928
         depth18           0.970261         0.461867  0.444920
         depth19           0.944397         0.333333  0.314799
         depth20           0.937500         0.333333  0.312500
image 59 depth1            0.981201         0.333333  0.327067
         depth2            0.965195         0.333333  0.321732
         depth3            0.958008         0.589360  0.439668
         depth4            0.969498         0.886392  0.716307
         depth5            0.971329         0.904190  0.767929
         depth6            0.968460         0.942209  0.760542
         depth7            0.973190         0.912757  0.801774
         depth8            0.948853         0.837108  0.684958
         depth9            0.934402         0.762802  0.579576
         depth10           0.942749         0.807517  0.624660
         depth11           0.942184         0.814737  0.616550
         depth12           0.956497         0.776095  0.631838
         depth13           0.955688         0.825597  0.653929
         depth14           0.961990         0.333333  0.320663
         depth15           0.967834         0.333333  0.322611

[175 rows x 3 columns]

In [110]:
data={'Total AVG Overall Precision':[Averages['AVG Overall Precision'].mean()[0]],\
      'Total AVG Per Class Precision':[Averages['AVG Macro Precision'].mean()[0]],\
      'Total AVG Iou':[Averages['AVG Iou'].mean()[0]]}
total_averages=pd.DataFrame(data=data,index=['Unet2d'])
total_averages

,Total AVG Overall Precision,Total AVG Per Class Precision,Total AVG Iou
Unet2d,0.950998,0.602768,0.463523
